<a href="https://colab.research.google.com/github/fermuba/Helper-IA/blob/main/notebooks/Transformacion_de_archivos_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformación y unificación de datasets HR — Hackathon Microsoft

Este notebook procesa y unifica dos archivos JSON de tickets de RR.HH., desanida la estructura de entidades y genera un dataset final listo para análisis y uso en modelos o Azure Functions.

## Origen de los datos
Los datos utilizados provienen del repositorio público:  
**SAP / hr-request-data-set**  
https://github.com/SAP/hr-request-data-set

En particular, se emplean los archivos:
- `data.json`
- `data_new.json`

Ambos forman parte del dataset de tickets simulados de Recursos Humanos, con campos como `ticket`, `category`, `sub_category`, `entities` y `label`.


In [ ]:
import pandas as pd
print(pd.__version__)
import json

# Ruta del archivo descargado
ruta_json = "/content/drive/MyDrive/Hackathon2025/data.json"

# Opción 1: leer directamente con pandas si es un JSON tipo records
try:
    df = pd.read_json(ruta_json)
except ValueError:
    # Opción 2: leer con json y luego pasar a DataFrame
    with open(ruta_json, "r", encoding="utf-8") as f:
        data = json.load(f)
    df = pd.DataFrame(data)

# Revisar las primeras filas
print(df.head())
# Ver columnas y tipo de datos
print(df.info())


   id                                             ticket    category  \
0   1  Dear HR, I'm Virginie Thibault from the admini...      Salary   
1   2  Dear Sir./Madam., this email is to make you aw...   Complaint   
2   3  Dear colleagues, \nI need to request a time of...  Life event   
3   4  Dear colleagues, \n\nthere is a personal issue...   Complaint   
4   5  Dear colleagues, \nI want to get a refund for ...      Refund   

      sub_category                                           entities  \
0     Salary raise  [[197, 199, increase_in_percentage], [77, 115,...   
1        complaint          [[163, 205, complaint], [83, 94, to_who]]   
2  Personal issues                [[76, 118, description_life_event]]   
3        complaint        [[336, 391, complaint], [154, 170, to_who]]   
4    Refund travel  [[67, 76, location], [80, 86, location], [95, ...   

                        label  
0         Salary_Salary raise  
1         Complaint_complaint  
2  Life event_Personal issues  


In [ ]:
df.head()

,id,ticket,category,sub_category,entities,label
0,1,"Dear HR, I'm Virginie Thibault from the admini...",Salary,Salary raise,"[[197, 199, increase_in_percentage], [77, 115,...",Salary_Salary raise
1,2,"Dear Sir./Madam., this email is to make you aw...",Complaint,complaint,"[[163, 205, complaint], [83, 94, to_who]]",Complaint_complaint
2,3,"Dear colleagues, \nI need to request a time of...",Life event,Personal issues,"[[76, 118, description_life_event]]",Life event_Personal issues
3,4,"Dear colleagues, \n\nthere is a personal issue...",Complaint,complaint,"[[336, 391, complaint], [154, 170, to_who]]",Complaint_complaint
4,5,"Dear colleagues, \nI want to get a refund for ...",Refund,Refund travel,"[[67, 76, location], [80, 86, location], [95, ...",Refund_Refund travel


In [ ]:
# Ruta del archivo descargado
ruta_json = "/content/drive/MyDrive/Hackathon2025/data_new.json"

# Opción 1: leer directamente con pandas si es un JSON tipo records
try:
    df_new = pd.read_json(ruta_json)
except ValueError:
    # Opción 2: leer con json y luego pasar a DataFrame
    with open(ruta_json, "r", encoding="utf-8") as f:
        data = json.load(f)
    df_new = pd.DataFrame(data)

df_new.head()

,id,ticket,category,sub_category,entities,label
0,1,"Dear HR, I'm Virginie Thibault from the admini...",Salary,Salary raise,"[[197, 199, increase_in_percentage], [77, 115,...",Salary_Salary raise
1,2,"Dear Sir./Madam., this email is to make you aw...",Complaint,complaint,"[[163, 205, complaint], [83, 94, to_who]]",Complaint_complaint
2,3,"Dear colleagues, \nI need to request a time of...",Life event,Personal issues,"[[75, 117, description_life_event]]",Life event_Personal issues
3,4,"Dear colleagues, \n\nthere is a personal issue...",Complaint,complaint,"[[333, 388, complaint], [153, 171, to_who]]",Complaint_complaint
4,5,"Dear colleagues, \nI want to get a refund for ...",Refund,Refund travel,"[[67, 76, location], [80, 86, location], [94, ...",Refund_Refund travel


In [ ]:
# Tamaños de df y df_new
print('='*80)
print(f'El archivo data.json esta en el dataframe df y tiene {df.shape[0]} filas y {df.shape[1]} columnas')
print(f'El archivo data_new.json esta en el dataframe df_new y tiene {df_new.shape[0]} filas y {df_new.shape[1]} columnas')
print('='*80)


El archivo data.json esta en el dataframe df y tiene 259 filas y 6 columnas
El archivo data_new.json esta en el dataframe df_new y tiene 259 filas y 6 columnas


In [ ]:
# Unir los dataframe
df_full = pd.concat([df,df_new], ignore_index=True)
print('El nuevo tamaño es: ', df_full.shape)

El nuevo tamaño es:  (518, 6)


In [ ]:
# Aplicar transformaciones
# 1. Expande la lista: cada entidad se convierte en una fila
df_expanded = df_full.explode("entities").reset_index(drop=True)

# Filtrar las filas donde 'entities' no es una lista válida (ej. NaN después de explode)
df_expanded_filtered = df_expanded[df_expanded['entities'].apply(lambda x: isinstance(x, list) or pd.isna(x))].copy()

# Filtrar NaN generados por explode de listas vacías, que son el origen del error
df_expanded_filtered = df_expanded_filtered[df_expanded_filtered['entities'].notna()]

# 2. Convierte cada entidad (lista) en columnas
df_entities = pd.DataFrame(df_expanded_filtered["entities"].tolist(),
                           columns=["entity_start", "entity_end", "entity_label"])

# 3. Une las nuevas columnas al dataframe original (ya expandido y filtrado)
df_final = pd.concat([df_expanded_filtered.drop(columns=["entities"]), df_entities], axis=1)

# 4. Opcional: ordena las columnas
df_final = df_final[["id", "ticket", "category", "sub_category",
                     "entity_start", "entity_end", "entity_label", "label"]]

# 5. Convertir columnas flotante a entero
df_final['entity_start'] = df_final['entity_start'].astype('Int64')
df_final['entity_end'] = df_final['entity_end'].astype('Int64')
df_final['id'] = df_final['id'].astype('Int64')


df_final.head()

,id,ticket,category,sub_category,entity_start,entity_end,entity_label,label
0,1,"Dear HR, I'm Virginie Thibault from the admini...",Salary,Salary raise,197,199,increase_in_percentage,Salary_Salary raise
1,1,"Dear HR, I'm Virginie Thibault from the admini...",Salary,Salary raise,77,115,work_title,Salary_Salary raise
2,1,"Dear HR, I'm Virginie Thibault from the admini...",Salary,Salary raise,238,249,salary,Salary_Salary raise
3,1,"Dear HR, I'm Virginie Thibault from the admini...",Salary,Salary raise,253,264,salary,Salary_Salary raise
4,2,"Dear Sir./Madam., this email is to make you aw...",Complaint,complaint,163,205,complaint,Complaint_complaint


In [ ]:

df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1047 entries, 0 to 926
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            999 non-null    Int64 
 1   ticket        999 non-null    object
 2   category      999 non-null    object
 3   sub_category  999 non-null    object
 4   entity_start  999 non-null    Int64 
 5   entity_end    999 non-null    Int64 
 6   entity_label  999 non-null    object
 7   label         999 non-null    object
dtypes: Int64(3), object(5)
memory usage: 76.7+ KB


In [ ]:
# Ejemplode ticket
df_final['ticket'][0]

"Dear HR, I'm Virginie Thibault from the administration office and worikng as secretary and administrative assistant. I would like to ask you whether is it possible to have a salary increase of the 9%, so to pass from my current salary of 45400 euros to 49500 euros. Thank you in advance for your comprehension and your kind response. Best regards, VT."

In [ ]:
# Registros duplicados
df_final.duplicated().sum()

np.int64(27)

In [ ]:
df_final[df_final.duplicated()]

,id,ticket,category,sub_category,entity_start,entity_end,entity_label,label
1000,239,"Dear sir, madame, I would like to ask for a re...",Refund,Refund travel,<NA>,<NA>,NaN,Refund_Refund travel
1001,239,"Dear sir, madame, I would like to ask for a re...",Refund,Refund travel,<NA>,<NA>,NaN,Refund_Refund travel
1002,239,"Dear sir, madame, I would like to ask for a re...",Refund,Refund travel,<NA>,<NA>,NaN,Refund_Refund travel
1003,239,"Dear sir, madame, I would like to ask for a re...",Refund,Refund travel,<NA>,<NA>,NaN,Refund_Refund travel
1004,239,"Dear sir, madame, I would like to ask for a re...",Refund,Refund travel,<NA>,<NA>,NaN,Refund_Refund travel
1006,240,"Dear sir, madame, I would like to report a col...",Complaint,complaint,<NA>,<NA>,NaN,Complaint_complaint
1008,241,"To whom it may concern, I will be on a 6 month...",Ask information,Accommodation,<NA>,<NA>,NaN,Ask information_Accommodation
1010,242,"Dear HR service, I would like to request a sic...",Life event,Health issues,<NA>,<NA>,NaN,Life event_Health issues
1012,243,"Hello, I will need to spend a month in our off...",Ask information,Accommodation,<NA>,<NA>,NaN,Ask information_Accommodation
1015,245,"Dear Sir/Madam, I recently have been discrimin...",Complaint,complaint,<NA>,<NA>,NaN,Complaint_complaint


In [ ]:
# Eliminar duplicados
df_final = df_final.drop_duplicates()

df_final.duplicated().sum()

np.int64(0)

In [ ]:
# Total de registros nulos por columna
df_final.isna().sum()

,0
id,48
ticket,48
category,48
sub_category,48
entity_start,21
entity_end,21
entity_label,21
label,48


In [ ]:
# Eliminación de registros nulos
df_final.dropna(inplace=True)
df_final = df_final.reset_index(drop=True)

df_final.isna().sum()

,0
id,0
ticket,0
category,0
sub_category,0
entity_start,0
entity_end,0
entity_label,0
label,0


In [ ]:
# Nuevo tañano despues de limpieza
print('='*80)
print(f'El dataframe limpio tiene {df_final.shape[0]} filas y {df_final.shape[1]} columnas')
print('='*80)


El dataframe limpio tiene 951 filas y 8 columnas


In [ ]:
# Exportar archivo limpio en csv
df_final.to_csv("/content/drive/MyDrive/Hackathon2025/data_clean.csv", index=False)

In [ ]:
# Exportar archivo limpio en formato JSON (records) y descargarlo para Azure Cognitive Search
df_final.to_json("data_clean.json", orient="records", force_ascii=False)
files.download("data_clean.json")